##### Code to preprocess / Organize and clean ProviApt LOG Data

In [8]:
import numpy as np
import pandas as pd
import os
import re
from datetime import date

In [9]:
import warnings    # to avoid warning during executions
warnings.filterwarnings("ignore")

In [24]:
# append a line to previous line if its not starting in stanard format
with open('..\..\dataExport\LOGdata\log_2022_4_20.csv', 'r+', encoding="utf-8") as file:
    text = str();
    for line in file:
        if line[0:3] == "202":
            text = text + '\n';
        text = text + line.strip();
    file.seek(0);
    file.write(text);

In [28]:
log_rawDF = pd.read_csv('..\..\dataExport\LOGdata\log_2022_4_20.csv', sep='*')

In [29]:
log_rawDF.head(50)

,Date_Time,Time,MicroSec,Type,Unit,Parameter,Message,Value1,Value2
0,2021-4-20 13:0:13,1618916413,613129,4,FPC11_FPP11,Arduino,"more than 2 communication restarts, try powerd...",NaN,NaN
1,2021-4-20 13:0:57,1618916457,9327,4,FPC11_FPP11,Arduino,"more than 2 communication restarts, try powerd...",NaN,NaN
2,2021-4-20 13:1:38,1618916498,812633,4,FPC11_FPP11,Arduino,"more than 2 communication restarts, try powerd...",NaN,NaN
3,2021-4-20 13:2:20,1618916540,214501,4,FPC11_FPP11,Arduino,"more than 2 communication restarts, try powerd...",NaN,NaN
4,2021-4-20 13:3:3,1618916583,212528,4,FPC11_FPP11,Arduino,"more than 2 communication restarts, try powerd...",NaN,NaN
5,2021-4-20 13:3:46,1618916626,209680,4,FPC11_FPP11,Arduino,"more than 2 communication restarts, try powerd...",NaN,NaN
6,2021-4-20 13:3:58,1618916638,271269,3,FPC14,program,step ID - Changed (new and old value),4_WaterAcid,5_empty
7,2021-4-20 13:4:1,1618916641,101959,3,FPC14,program,start at - Changed (new and old value),sellected step at...,first step
8,2021-4-20 13:4:6,1618916646,386245,3,FPC14,program,procedure running - Changed,not active,active
9,2021-4-20 13:4:7,1618916647,639887,2,FPC14,program,Program F_ThalaPseu_HalfNutr started,NaN,NaN


In [30]:
print(log_rawDF.shape)

(865668, 9)


In [31]:
print(log_rawDF.isnull().values.any())
print(log_rawDF.isnull().sum().sum())

True
824890


In [32]:
log_rawDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 865668 entries, 0 to 865667
Data columns (total 9 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   Date_Time  865668 non-null  object
 1   Time       865668 non-null  int64 
 2   MicroSec   865668 non-null  int64 
 3   Type       865668 non-null  int64 
 4   Unit       865668 non-null  object
 5   Parameter  864368 non-null  object
 6   Message    865668 non-null  object
 7   Value1     411877 non-null  object
 8   Value2     495869 non-null  object
dtypes: int64(3), object(6)
memory usage: 59.4+ MB


In [33]:
log_rawDF['Parameter'].value_counts()

T10                 203861
Arduino             102024
feeding              80955
calculations         78184
AutoTemplateSync     49465
                     ...  
waste                    2
T41                      2
T44                      2
T45                      2
V42                      2
Name: Parameter, Length: 162, dtype: int64

In [34]:
log_rawDF['Parameter'].nunique()

162

In [35]:
log_parameters = log_rawDF['Parameter'].unique()
print(log_parameters)

['Arduino' 'program' 'T61' 'water' 'T10' 'Reactor' 'T71' 'pneumatic'
 'autoSyncMonitor' 'T42A' 'harvest' 'calculations' 'P1L' 'feeding' 'T64'
 'T42' 'T53' 'T51' 'T73' 'T74' 'T65' 'T63' 'T72' 'T52' 'T54' 'PT1_V4'
 'T55' 'T43' 'LEDlighting' 'RM29' 'RM6' 'V74' nan 'RM12' 'RM13' 'RM14'
 'RM15' 'RM16' 'RM17' 'RM19' 'RM2' 'RM21' 'RM22' 'RM23' 'T41A' 'T43A'
 'RM35' 'RM4' 'RM5' 'RM7' 'P42' 'RM1' 'RM30' 'Arduino_QG' 'Arduino_pH'
 'V3' 'P1R' 'P2R' 'CO2_V2' 'R1' 'PTdiff_K2' 'L1W' 'Arduino_TT1'
 'Arduino_LT1' 'Arduino1_LT52' 'Arduino2_LT72' 'Arduino1_LT41'
 'Arduino2_LT61' 'AutoLink' 'P43' 'V42' 'P10' 'V11' 'V8' 'LT1' 'RM3'
 'RM11' 'RM8' 'M1' 'P2L' 'V61' 'P11' 'Recipes' 'Products' 'L1' 'V6' 'V1'
 'V72' 'chiller' 'T62' 'V71' 'V62' 'P12' 'V43' 'L1FR' 'L1R' 'L1B' 'LT10'
 'M10' 'V63' 'RM18' 'Arduino1' 'V52' 'V64' 'PT10' 'TT0' 'waste' 'V51'
 'RM20' 'RM33' 'AutoTemplateSync' 'Arduino2_LT73' 'Arduino_CO2'
 'Arduino2_LT63' 'Arduino2_LT65' 'Arduino1_LT53' 'Arduino2_LT64'
 'Arduino2_LT75' 'Arduino1_LT54' 'A

In [36]:
log_parameters = log_rawDF['Type'].unique()
print(log_parameters)

[4 3 2 5 6 1]


In [37]:
log_rawDF['Type'].value_counts()

2    516084
1    202503
4     67915
6     49397
5     18991
3     10778
Name: Type, dtype: int64

In [38]:
log_rawDF['Type'].isna().sum()

0

In [39]:
log_rawDF[log_rawDF['Type'].isnull()].index.tolist()

[]

In [40]:
log_parameters = log_rawDF['Unit'].unique()
print(log_parameters)
print(log_rawDF['Unit'].nunique())

['FPC11_FPP11' 'FPC14' 'FFU' 'FPC22' 'FPC21_AUT21' 'FPC21' 'FPC23' 'FPC13'
 'FPC24' 'FPC31' 'FPC23_FPP23' 'FPC41' 'FPC42' 'FPC11' 'Product'
 '&lt;EVAL&gt;' 'FPC21_FPP21' 'FPC13_FPP13' 'FPC24_FPP24' 'FFP_FFP'
 'FPC31_FPP31' 'FPC22_FPP22' 'FPC12' 'FPP11' 'FPC14_FPP14' 'FPP21' 'FPP13'
 'FPP14' 'FPP' 'FPC12_FPP' 'FPP31' 'docProduction' 'Dashboard' 'FFP_AUTFU'
 'FPC13_AUT13' 'FPC22_AUT22' 'FPC11_AUT11' 'FPC12_AUT' 'FPC31_AUT31'
 'FPC24_AUT24' 'FPC23_AUT23' 'FFU_harvestTank_DoseRequests'
 'FFU_T10_RecipeRequests' 'scheduler' 'ProductionTanks'
 'harvestTank-TotalTime' 'water-TotalTime' 'RM36-TotalTime'
 'AllComponentTanks' 'T10-TotalTime' 'cntrTankDetail' 'T71-TotalTime'
 'RM14-TotalTime' 'RM11-TotalTime' 'Reactor-TotalTime' 'RM38-TotalTime'
 'T64-TotalTime' 'RM31-TotalTime' 'RM3-TotalTime'
 'docPoduction-FF-TotalTime' 'RM37-TotalTime' 'T43A-TotalTime'
 'RM6-TotalTime' 'RM23-TotalTime' 'RM1-TotalTime' 'ComponentTanks'
 'RawMaterialTanks' 'FPC14_AUT14' 'dashboard']
69


#### Data Frame without debug messages

In [41]:
logNoDBG = log_rawDF.query('Type != 1')

In [42]:
logNoDBG.shape

(663165, 9)

In [43]:
logNoDBG_type = logNoDBG['Type'].unique()
print(logNoDBG_type)

[4 3 2 5 6]


In [44]:
logNoDBG_unit = logNoDBG['Unit'].unique()
print(logNoDBG_unit)
print(logNoDBG['Unit'].nunique())

['FPC11_FPP11' 'FPC14' 'FFU' 'FPC22' 'FPC21_AUT21' 'FPC21' 'FPC23' 'FPC13'
 'FPC24' 'FPC31' 'FPC23_FPP23' 'FPC41' 'FPC42' 'FPC11' 'Product'
 '&lt;EVAL&gt;' 'FPC21_FPP21' 'FPC13_FPP13' 'FPC24_FPP24' 'FFP_FFP'
 'FPC31_FPP31' 'FPC22_FPP22' 'FPC12' 'FPP11' 'FPC14_FPP14' 'FPP21' 'FPP13'
 'FPP14' 'FPP' 'FPC12_FPP' 'FPP31' 'docProduction' 'FPC23_AUT23']
33


In [45]:
logNoDBG_parameters = logNoDBG['Parameter'].unique()
print(logNoDBG_parameters)
print(logNoDBG['Parameter'].nunique())

['Arduino' 'program' 'T61' 'water' 'T10' 'Reactor' 'T71' 'pneumatic'
 'autoSyncMonitor' 'T42A' 'harvest' 'calculations' 'P1L' 'feeding' 'T64'
 'T42' 'T53' 'T51' 'T73' 'T74' 'T65' 'T63' 'T72' 'T52' 'T54' 'PT1_V4'
 'T55' 'T43' 'LEDlighting' 'RM29' 'RM6' 'V74' nan 'RM12' 'RM13' 'RM14'
 'RM15' 'RM16' 'RM17' 'RM19' 'RM2' 'RM21' 'RM22' 'RM23' 'T41A' 'T43A'
 'RM35' 'RM4' 'RM5' 'RM7' 'P42' 'RM1' 'RM30' 'Arduino_QG' 'Arduino_pH'
 'V3' 'P1R' 'P2R' 'CO2_V2' 'R1' 'PTdiff_K2' 'L1W' 'Arduino_TT1'
 'Arduino_LT1' 'Arduino1_LT52' 'Arduino2_LT72' 'Arduino1_LT41'
 'Arduino2_LT61' 'AutoLink' 'P43' 'V42' 'P10' 'V11' 'V8' 'LT1' 'RM3'
 'RM11' 'RM8' 'M1' 'P2L' 'V61' 'P11' 'Recipes' 'Products' 'L1' 'V6' 'V1'
 'V72' 'chiller' 'T62' 'V71' 'V62' 'P12' 'V43' 'L1FR' 'L1R' 'L1B' 'LT10'
 'M10' 'V63' 'RM18' 'Arduino1' 'V52' 'V64' 'PT10' 'TT0' 'waste' 'V51'
 'RM20' 'RM33' 'Arduino_FT2' 'Arduino_CO2']
111


In [46]:
logNoDBG.isnull().sum()

Date_Time         0
Time              0
MicroSec          0
Type              0
Unit              0
Parameter       330
Message           0
Value1       255471
Value2       171701
dtype: int64

In [47]:
print(logNoDBG['Type'].isnull().values.any())
print(logNoDBG['Type'].isnull().sum().sum())

False
0


In [48]:
logNoDBG.head(50)

,Date_Time,Time,MicroSec,Type,Unit,Parameter,Message,Value1,Value2
0,2021-4-20 13:0:13,1618916413,613129,4,FPC11_FPP11,Arduino,"more than 2 communication restarts, try powerd...",NaN,NaN
1,2021-4-20 13:0:57,1618916457,9327,4,FPC11_FPP11,Arduino,"more than 2 communication restarts, try powerd...",NaN,NaN
2,2021-4-20 13:1:38,1618916498,812633,4,FPC11_FPP11,Arduino,"more than 2 communication restarts, try powerd...",NaN,NaN
3,2021-4-20 13:2:20,1618916540,214501,4,FPC11_FPP11,Arduino,"more than 2 communication restarts, try powerd...",NaN,NaN
4,2021-4-20 13:3:3,1618916583,212528,4,FPC11_FPP11,Arduino,"more than 2 communication restarts, try powerd...",NaN,NaN
5,2021-4-20 13:3:46,1618916626,209680,4,FPC11_FPP11,Arduino,"more than 2 communication restarts, try powerd...",NaN,NaN
6,2021-4-20 13:3:58,1618916638,271269,3,FPC14,program,step ID - Changed (new and old value),4_WaterAcid,5_empty
7,2021-4-20 13:4:1,1618916641,101959,3,FPC14,program,start at - Changed (new and old value),sellected step at...,first step
8,2021-4-20 13:4:6,1618916646,386245,3,FPC14,program,procedure running - Changed,not active,active
9,2021-4-20 13:4:7,1618916647,639887,2,FPC14,program,Program F_ThalaPseu_HalfNutr started,NaN,NaN
